# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

#Libs imported to run/integrate N-gram analysis with language model predictions
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [148]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".")
        
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)

        # Implement Bayesian approach to calculate posterior probabilities of each letter
        remaining_letters = [char for char in 'abcdefghijklmnopqrstuvwxyz' if char not in self.guessed_letters]
        letter_probability = {}
        for letter in remaining_letters:
            letter_count = full_dict_string.count(letter)
            total_letters = len(full_dict_string)
            # Add Bayesian smoothing (Laplace smoothing) to handle unseen letters
            probability = (letter_count + 1) / (total_letters + len(remaining_letters))
            letter_probability[letter] = probability
        
        # Choose the letter with the highest posterior probability as the most optimal guess
        bayesian_guess_letter = max(letter_probability, key=letter_probability.get)
        
        # Implement letter frequency and pattern recognition approach
        # Prioritize letters based on their frequency in the English language
        frequency_ordered_letters = 'etaoinshrdlcumwfgypbvkjxqz'
        
        # Consider common letter patterns (e.g., 'th', 'ing', 'er')
        common_letter_patterns = ['th', 'he', 'in', 'er', 'an', 're', 'ed', 'on', 'es', 'st']
        
        # Sort remaining letters based on their frequency in English language and common letter patterns
        sorted_letters = sorted(remaining_letters, key=lambda x: (frequency_ordered_letters.index(x), -sum(pattern in self.current_dictionary for pattern in common_letter_patterns)))
        
        # Choose the first letter from the sorted list as the most optimal guess
        frequency_guess_letter = sorted_letters[0] if sorted_letters else '!'

        # Choose the letter with the higher probability between Bayesian and frequency approaches
        guess_letter = bayesian_guess_letter if letter_probability.get(bayesian_guess_letter, 0) >= letter_probability.get(frequency_guess_letter, 0) else frequency_guess_letter
        
        return guess_letter        

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [5]:
api = HangmanAPI(access_token="d157db3d851d5495b13d051d3626e0", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [153]:
loop = 20
i = 1
while i <= loop:
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print(i)
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
    i =+ i+1

Successfully start a new game! Game ID: c93782ab7ec9. # of tries remaining: 6. Word: _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ '}
Guessing letter: r
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ _ '}
Guessing letter: s
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ _ _ _ _ '}
Guessing letter: a
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e _ _ _ _ _ '}
Guessing letter: n
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e n _ n _ _ '}
Guessing letter: o
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ e n o n _ o '}
Guessing letter: i
Sever response: {'game_id': 'c93782ab7ec9', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ e n 

Sever response: {'game_id': '241b948862b2', 'status': 'ongoing', 'tries_remains': 1, 'word': 'e _ a _ _ a t e d '}
Guessing letter: v
Sever response: {'game_id': '241b948862b2', 'status': 'ongoing', 'tries_remains': 1, 'word': 'e v a _ _ a t e d '}
Guessing letter: u
Sever response: {'game_id': '241b948862b2', 'status': 'ongoing', 'tries_remains': 1, 'word': 'e v a _ u a t e d '}
Guessing letter: o
Sever response: {'game_id': '241b948862b2', 'status': 'failed', 'tries_remains': 0, 'word': 'e v a _ u a t e d '}
Failed game: 241b948862b2. Because of: # of tries exceeded!
5
run 285 practice games out of an allotted 100,000. practice success rate so far = 0.196
Successfully start a new game! Game ID: a39880d943cb. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'a39880d943cb', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ _ e _ '}
Guessing letter: r
Sever response: {'game_id': 'a39880d943cb', 'status': 'ongoi

Sever response: {'game_id': '0e8c68e926cf', 'status': 'ongoing', 'tries_remains': 4, 'word': 't a r _ l e _ '}
Guessing letter: i
Sever response: {'game_id': '0e8c68e926cf', 'status': 'ongoing', 'tries_remains': 3, 'word': 't a r _ l e _ '}
Guessing letter: n
Sever response: {'game_id': '0e8c68e926cf', 'status': 'ongoing', 'tries_remains': 2, 'word': 't a r _ l e _ '}
Guessing letter: o
Sever response: {'game_id': '0e8c68e926cf', 'status': 'ongoing', 'tries_remains': 1, 'word': 't a r _ l e _ '}
Guessing letter: c
Sever response: {'game_id': '0e8c68e926cf', 'status': 'failed', 'tries_remains': 0, 'word': 't a r _ l e _ '}
Failed game: 0e8c68e926cf. Because of: # of tries exceeded!
9
run 289 practice games out of an allotted 100,000. practice success rate so far = 0.194
Successfully start a new game! Game ID: b5f4b9e75ef3. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'b5f4b9e75ef3', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ 

Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ _ '}
Guessing letter: i
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ i _ _ _ _ '}
Guessing letter: r
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ r i _ _ _ _ '}
Guessing letter: t
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 4, 'word': 't r i _ _ _ _ '}
Guessing letter: l
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 3, 'word': 't r i _ _ _ _ '}
Guessing letter: o
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 3, 'word': 't r i _ o _ o '}
Guessing letter: n
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 3, 'word': 't r i _ o n o '}
Guessing letter: s
Sever response: {'game_id': '4d8c583647cf', 'status': 'ongoing', 'tries_remains': 2, 'word

Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ e _ _ e _ '}
Guessing letter: s
Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ s e _ _ e _ '}
Guessing letter: r
Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ s e _ _ e _ '}
Guessing letter: d
Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ s e _ _ e d '}
Guessing letter: n
Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ s e _ _ e d '}
Guessing letter: t
Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ t _ s e _ t e d '}
Guessing letter: i
Sever response: {'game_id': 'fa85aa9686c6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ t i s e _ t e d '}
Guessing letter: a
Sever response: {'game_id': 'fa85a

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

In [123]:
import pandas as pd

# Load the word list from the file
word_list_file = 'words_250000_train.txt'
word_list = pd.read_csv(word_list_file, header=None, names=['word'])

# Preprocess the data (if needed)
# For example, you might want to convert all words to lowercase or remove any special characters
word_list['word'] = word_list['word'].str.lower()


In [124]:
word_list.dtypes

word    object
dtype: object

In [129]:
word_list['word'] = word_list['word'].astype(str)

In [130]:
word_list['word']

0             aaa
1          aaaaaa
2            aaas
3          aachen
4            aaee
           ...   
227295     zythia
227296    zyzomys
227297    zyzzyva
227298         zz
227299        zzz
Name: word, Length: 227300, dtype: object

In [134]:
# Extract features
word_list = word_list.dropna()  # Remove any rows with NaN values
word_list = word_list[word_list['word'].apply(lambda x: isinstance(x, str))]  # Remove any non-string values
word_list['word_length'] = word_list['word'].apply(len).astype(int)

# Sample a subset of the data
sampled_word_list = word_list.sample(n=10000, random_state=42)  # Adjust the number of samples as needed

# Separate features (X) and target (y)
X = sampled_word_list[['word_length']]
y = sampled_word_list['word']


In [135]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [136]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize and train the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

# Predict the target values on the testing set
y_pred = random_forest_model.predict(X_test)

# Evaluate the performance of the model using accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.0


In [144]:
import pandas as pd
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

class HangmanModel:
    def __init__(self, sample_size=10000):
        self.word_list = pd.read_csv('words_250000_train.txt', header=None, names=['word'])
        self.word_list = self.word_list.sample(n=sample_size, random_state=42)
        self.word_list['length'] = self.word_list['word'].apply(self.get_word_length)
        self.word_list['unique_letters'] = self.word_list['word'].apply(lambda x: len(set(x)))
        self.word_list['letter_counts'] = self.word_list['word'].apply(Counter)

    def get_word_length(self, word):
        try:
            return len(str(word))
        except Exception as e:
            print(f"Error processing word: {word}. Error: {e}")
            return None

    def guess_letter(self, guessed_letters, revealed_word):
        revealed_word = revealed_word.replace('_', '.')
        filtered_words = self.word_list[self.word_list['length'] == len(revealed_word)]
        for letter in guessed_letters:
            filtered_words = filtered_words[~filtered_words['word'].str.contains(letter)]
        for idx, letter in enumerate(revealed_word):
            if letter != '.':
                filtered_words = filtered_words[filtered_words['word'].apply(lambda x: x[idx] == letter)]
        letter_count = filtered_words['letter_counts'].apply(lambda x: sum(x.values()))
        guess_letter = letter_count.idxmax()
        return guess_letter

    def train(self):
        X_train, X_test, y_train, y_test = train_test_split(self.word_list[['length', 'unique_letters']], self.word_list['word'], test_size=0.2, random_state=42)
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)

    def evaluate(self):
        # Generate a sample test dataset
        sample_test_size = 1000
        sample_test = self.word_list.sample(n=sample_test_size, random_state=42)
        X_test = sample_test[['length', 'unique_letters']]
        y_test = sample_test['word']
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy

# Example usage:
hangman_model = HangmanModel(sample_size=10000)
hangman_model.train()
accuracy = hangman_model.evaluate()
print("Accuracy:", accuracy)


Accuracy: 0.0
